In [45]:
from nano_graphrag import GraphRAG, QueryParam
from examples.using_deepseek_as_llm import (
    deepseepk_model_if_cache,   # LLM ya listo y con cache opcional
    local_st_embeddings,
    WORKING_DIR,
)
from IPython.display import Markdown, display

# === Inicializa GraphRAG (ya indexaste antes con el mismo WORKING_DIR) ===
rag = GraphRAG(
    working_dir=WORKING_DIR,
    best_model_func=deepseepk_model_if_cache,
    cheap_model_func=deepseepk_model_if_cache,
    embedding_func=local_st_embeddings,
    enable_naive_rag=True,
    enable_llm_cache=False,
)

def show_md(text: str):
    display(Markdown(f"```\n{text.strip()}\n```"))

# === Naive (solo embeddings) ===
async def ask_naive(q: str, max_tokens: int = 180):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    # OJO: no se pasan kwargs aquí; GraphRAG no acepta temperature/max_tokens.
    return await rag.aquery(prompt, QueryParam(mode="naive"))

# === GraphRAG Global (comunidades) ===
async def ask_global(q: str, max_tokens: int = 180):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    return await rag.aquery(prompt, QueryParam(mode="global"))

# === Puro LLM (sin retrieval) usando la función del example ===
async def ask_llm_only(q: str, temperature: float = 0.7, max_tokens: int = 200):
    prompt = f"{q}\n\nResponde en máximo 5 líneas claras y concisas."
    # Esta función ya llama a DeepSeek con AsyncOpenAI y maneja cache si lo activas
    return await deepseepk_model_if_cache(
        prompt,
        temperature=temperature,
        max_tokens=max_tokens,
    )



INFO:nano-graphrag:Loading tokenizer: type='tiktoken', name='gpt-4o'
INFO:nano-graphrag:Load KV full_docs with 1 data
INFO:nano-graphrag:Load KV text_chunks with 42 data
INFO:nano-graphrag:Load KV community_reports with 98 data
/opt/miniconda3/envs/graphrag/lib/python3.11/xml/etree/ElementTree.py:581: RuntimeWarning: coroutine 'GraphRAG.ainsert' was never awaited
  self._root = parser._parse_whole(source)
INFO:nano-graphrag:Loaded graph from ./nano_graphrag_cache_deepseek_TEST/graph_chunk_entity_relation.graphml with 569 nodes, 601 edges
INFO:nano-vectordb:Load (556, 384) data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': './nano_graphrag_cache_deepseek_TEST/vdb_entities.json'} 556 data
INFO:nano-vectordb:Load (42, 384) data
INFO:nano-vectordb:Init {'embedding_dim': 384, 'metric': 'cosine', 'storage_file': './nano_graphrag_cache_deepseek_TEST/vdb_chunks.json'} 42 data


In [ ]:
# === Ejemplo de comparación ===
question = "Quien es Guzmán?"

ans_naive  = await ask_naive(question)
ans_global = await ask_global(question)
ans_llm    = await ask_llm_only(question, temperature=0.5, max_tokens=180)

print("NAIVE RAG:\n");   show_md(ans_naive)
print("NAIVE RAG:\n");   show_md(ans_naive)
print("\nGRAPH RAG (GLOBAL):\n"); show_md(ans_global)
print("\nPURO LLM:\n");    show_md(ans_llm)

Batches: 100%|██████████| 1/1 [00:00<00:00, 24.99it/s]
INFO:nano-graphrag:Revtrieved 96 communities
INFO:nano-graphrag:Grouping to 5 groups for global search
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:nano-graphrag:JSON data successfully extracted.
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https:/

🧩 NAIVE RAG:



```
Sorry, I'm not able to provide an answer to that question.
```


🌐 GRAPH RAG (GLOBAL):



```
Basado en la información disponible, el profesor Guzmán era un académico que mantenía un estudio en el mismo edificio que la oficina de Scrooge en la Universidad Nacional de Colombia, campus Medellín. Esto indica que desarrollaba sus actividades en un entorno universitario.
```


🧠 PURO LLM:



```
Guzmán es un nombre que puede referirse a varias figuras históricas y contemporáneas. La más conocida internacionalmente es **Joaquín "El Chapo" Guzmán**, el narcotraficante mexicano fundador del Cártel de Sinaloa, quien fue capturado y extraditado a Estados Unidos. También puede hacer referencia a **Luis Guzmán**, un actor puertorriqueño-estadounidense conocido por sus papeles en cine y televisión. En un contexto histórico, **Alonso Pérez de Guzmán** fue un noble y militar español conocido como "Guzmán el Bueno". Sin más contexto, es imposible determinar a cuál se refiere específicamente.
```